In [2]:
# Import Libraries

import numpy as np
import pandas as pd
import adjacency
import os
import geopandas as gpd

In [3]:
DATA_PATH = '/home/h6x/git_projects/data_processing/processed_data/SVI/SVI2018_MIN_MAX_SCALED_MISSING_REMOVED_GNN_ID'
VARIABLE = 'EP_POV'

In [4]:
state= 'AL'
svi_od_path = os.path.join(DATA_PATH, state, state + '.shp')

In [5]:
svi_od = gpd.read_file(svi_od_path)

In [6]:
svi_od.head()

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,E_UNINSUR,M_UNINSUR,EP_UNINSUR,MP_UNINSUR,E_DAYPOP,Shape_Leng,Shape_Area,gnn_id,percentile,geometry
0,01,ALABAMA,AL,01001,Autauga,01001020100,"Census Tract 201, Autauga County, Alabama",3.790677,1923,253,...,178,79,0.093,4.1,1018,0.150082,0.000948,609,2,"POLYGON ((-86.50916 32.47344, -86.50620 32.475..."
1,01,ALABAMA,AL,01001,Autauga,01001020300,"Census Tract 203, Autauga County, Alabama",2.065365,3476,433,...,119,81,0.034,2.2,1836,0.100175,0.000516,610,2,"POLYGON ((-86.47087 32.47573, -86.46964 32.478..."
2,01,ALABAMA,AL,01001,Autauga,01001020400,"Census Tract 204, Autauga County, Alabama",2.464982,3831,337,...,108,100,0.028,2.6,1951,0.114106,0.000609,611,2,"POLYGON ((-86.45394 32.49318, -86.44742 32.493..."
3,01,ALABAMA,AL,01001,Autauga,01001020500,"Census Tract 205, Autauga County, Alabama",4.404987,9883,726,...,398,223,0.042,2.4,8342,0.159359,0.001099,612,2,"POLYGON ((-86.43816 32.45069, -86.43773 32.451..."
4,01,ALABAMA,AL,01001,Autauga,01001020801,"Census Tract 208.01, Autauga County, Alabama",47.981925,2826,324,...,144,84,0.051,2.9,1331,0.587644,0.012689,613,2,"POLYGON ((-86.59535 32.38272, -86.59454 32.383..."


In [7]:
svi_od['STCNTY'].unique()

array(['01001', '01003', '01005', '01007', '01009', '01013', '01015',
       '01017', '01021', '01025', '01027', '01031', '01033', '01035',
       '01039', '01043', '01045', '01047', '01049', '01051', '01053',
       '01055', '01057', '01059', '01063', '01067', '01069', '01071',
       '01073', '01075', '01077', '01079', '01081', '01083', '01087',
       '01089', '01091', '01093', '01095', '01097', '01099', '01101',
       '01103', '01107', '01109', '01111', '01113', '01115', '01117',
       '01121', '01123', '01125', '01127', '01133', '01011', '01019',
       '01023', '01029', '01037', '01041', '01061', '01065', '01085',
       '01105', '01119', '01129', '01131'], dtype=object)

In [8]:
svi_od.shape

(1174, 129)

In [15]:
graph_data = adjacency.process_county(state=state, county_stcnty='01043', variable=VARIABLE,data_path=DATA_PATH)

In [16]:
simplices = graph_data['simplices']

In [17]:
variable_df = graph_data['dataframe']

In [18]:
len(simplices)

83

In [22]:
p=0

edges = []
for i in range(len(simplices)):
    # print(simplices[i]
    
    if len(simplices[i])==2:

        edge = []
        
        for j in range(len(simplices[i])):

            # print(simplices[i][j])
            # print(variable_df[variable_df['simplices']==i])

            print(variable_df[variable_df['sortedID']==simplices[i][j]]['gnn_id'].values[0])
            edge.append(variable_df[variable_df['sortedID']==simplices[i][j]]['gnn_id'].values[0])

        print(edge)
        # flip the edge
        print(edge[::-1])

        edges.append(edge)

    # print(variable_df[variable_df['simplices']==i])

    # p=p+1
    # if p==4:


        break

45922
45923
[45922, 45923]
[45923, 45922]


In [14]:
edges

[]

In [14]:
import pandas as pd
import torch
import torch_geometric
from torch_geometric.data import Dataset, download_url
import numpy as np 
import os
from tqdm import tqdm
import os.path as osp

# import deepchem as dc
# from rdkit import Chem 

print(f"Torch version: {torch.__version__}")
print(f"Cuda available: {torch.cuda.is_available()}")
print(f"Torch geometric version: {torch_geometric.__version__}")

Torch version: 2.3.0+cu121
Cuda available: True
Torch geometric version: 2.5.3


In [38]:
class OpioidDataset(Dataset):
    def __init__(self, root,filename,test=False, transform=None, pre_transform=None, pre_filter=None):
        """
        root = Where the dataset should be stored. This folder is split
        into raw_dir (downloaded dataset) and processed_dir (processed data). 
        """
        self.test = test
        self.filename = filename
        super(OpioidDataset, self).__init__(root, transform, pre_transform)

    @property
    def raw_file_names(self):
        """ If this file exists in raw_dir, the download is not triggered.
            (The download func. is not implemented here)  
        """
        return self.filename

    @property
    def processed_file_names(self):
        """ If these files are found in raw_dir, processing is skipped"""
        self.data = pd.read_csv(self.raw_paths[0]).reset_index()

        if self.test:
            return [f'data_test_{i}.pt' for i in list(self.data.index)]
        else:
            return [f'data_{i}.pt' for i in list(self.data.index)]

    def download(self):
        pass

    def process(self):
        pass 
        # idx = 0
        # for raw_path in self.raw_paths:
        #     # Read data from `raw_path`.
        #     data = Data(...)

        #     if self.pre_filter is not None and not self.pre_filter(data):
        #         continue

        #     if self.pre_transform is not None:
        #         data = self.pre_transform(data)

        #     torch.save(data, osp.join(self.processed_dir, f'data_{idx}.pt'))
        #     idx += 1

    def _get_label(self, label):
        label = np.asarray([label])
        return torch.tensor(label, dtype=torch.int64)

    def len(self):
        return self.data.shape[0]

    def get(self, idx):
        """ - Equivalent to __getitem__ in pytorch
            - Is not needed for PyG's InMemoryDataset
        """
        if self.test:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_test_{idx}.pt'))
        else:
            data = torch.load(os.path.join(self.processed_dir, 
                                 f'data_{idx}.pt'))        
        return data

In [26]:
# us_svi = gpd.read_file('/home/h6x/git_projects/data_processing/raw_data/svi/2018/SVI2018_US_tract_dropped_missing.gdb')

In [27]:
# make us_svi['STCNTY'].unique() a list
county_names = us_svi['STCNTY'].unique().tolist()

In [31]:
# add '.npy' to each element in the list
county_names = [name + '.npy' for name in county_names]

In [32]:
len(county_names)

3141

In [33]:
county_names[0]

'01001.npy'

In [34]:
root_name = "/home/h6x/git_projects/gnn/model_1/data"

In [39]:
OpioidDataset(root_name, filename=county_names)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x93 in position 0: invalid start byte